In [1]:
import numpy as np
import pandas as pd

In [2]:
# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('./data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./data/u.data', sep='\t', names=r_cols, encoding='latin-1')

In [3]:
movies.head()

,movie_id,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
ratings

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [6]:
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [7]:
# train, test set 분리
from sklearn.model_selection import train_test_split

x = ratings.copy() # copy() 함수를 사용한 이유는 원본을 보본하기 위해서이다
y = ratings['user_id']
# 기존의 데이터를 train set과 test set으로 분리할 때, user_id를 기준하는 stratfied sampling 방식을 사용하기 위함이다. 
# 즉 기존 데이터에서의 각 user_id 별로 train set과 test set의 비율을 동일하게 유지하게 해준다 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)
# 실제 분리를 한다. 75% 데이터는 학습용 데이터 train set(x_train)에, 25%의 데이터는 정확도 검증용 데이터인 test set(x_test)에 저장
# y_train, y_test에는 사용자 di가 저장된다 

In [8]:
# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [9]:
# 모델별 RMSE를 계산하는 함수 
# 에측 모델(model)을 넘겨 받아서 해당 모델로 에측한 에측치의 정확도를 계산하는 함수 
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id']) # 에측 대상인 test set에 있는 사용자와 영화를 pair로 작을 맞춰 데이터를 만든다
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs]) # 모든 사용자-영화 짝에 대해서 주어진 모델에 의한 예측값을 계산
    y_true = np.array(x_test['rating']) # 실제 평점값 리스트를 x_test에서 받아온다 
    return RMSE(y_true, y_pred)

In [10]:
# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')
# pandas의 dataframe의 pivot() 메소드를 촬용해서 train set의 full matrix를 구한다. 
rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1667,1669,1671,1672,1673,1674,1675,1676,1678,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,4.0,NaN,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# 전체 평균으로 예측치를 계산하는 기본 모델

def best_seller(user_id, movie_id):
    '''
    이 함수는 평점을 예측할 대 모든 사용자의 평균(best-seller 방식)을 사용하는 함수이다. 
    user_id 하나와 movie_id 하나를 넘겨받아서 해당 사용자의 해당 영화에 대한 평점 에측치를 반환한다
    '''
    try:
        rating = train_mean[movie_id] # 해당 영화가 평균 데이터(train_mean)에 존재한다면 평균값을 돌려준다 
    except: 
        rating = 3.0 # 해당 영화가 평균 데이터에 존재하지 않으면 기본값(3.0)을 돌려준다 
    return rating

train_mean = x_train.groupby(['movie_id'])['rating'].mean() # best-seller 모델을 사용하기 위해서 영화의 전체(정확하게는 trains est)에 대한 평균을 구한다 
score(best_seller) # 

1.0227052261879865

In [12]:
# Full matrix를 사용자 데이터와 merge
# 학습용 데이터와 사용자 데이터를 합친다. merge() 함수는 공통의 이름을 가진 key 가 있으면 그것을 기준으로 합치기 때문에 현재 x_train과 users의 공통 key인 user_id를 기준으로 합친다
merged_ratings = pd.merge(x_train, users)
users = users.set_index('user_id') # user의 index를 user_id로 정한다. 나중에 사용자의 데이터를 확인하기 위해 필요 

In [13]:
# gender별 평점평균 계산
g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()
g_mean

movie_id  sex
1         F      3.788889
          M      3.895833
2         F      3.250000
          M      3.164557
3         F      2.333333
                   ...   
1674      M      4.000000
1675      M      3.000000
1676      M      2.000000
1678      M      1.000000
1682      M      3.000000
Name: rating, Length: 3036, dtype: float64

In [14]:
###### Gender기준 추천 ######
# gender별 평균을 예측치로 돌려주는 함수 
def cf_gender(user_id, movie_id):
    if movie_id in rating_matrix: # 예측치를 구하는 영화가 데이터에 있는지 확인
        gender = users.loc[user_id]['sex'] # 예측 대상인 사용자의 성별을 users에서 구해온다, 이것 때문에 users의 index를 user_id로 지정
        if gender in g_mean[movie_id]: # 예측 대상 영화가 해당 성별의 평균을 가지고 있는지 확인
            gender_rating = g_mean[movie_id][gender] # 해당 영화의 평균값을 에측치로 돌려준다 
        else:
            gender_rating = 3.0
    else:
        gender_rating = 3.0
    return gender_rating


In [15]:
score(cf_gender)

1.0324094188014417